# Block statements

In [4]:
int test1()
{// I'm in a block
    int x{4};
    {
        // I'm in a block, in a block
        int x{10};
        /* no variable conflict here, because `x` can be "shadowed"
           within a block. This `x` falls out of scope upon leaving
           the block
        */
    }
    return x;  // x == 4
}

# Global Scope

Variables declared outside of a block are called **global variables**. Global variables have **static duration**, which means they are created when the program starts and are destroyed when it ends. Global variables have **file scope** (also informally called “global scope” or “global namespace scope”), which means they are visible until the end of the file in which they are declared.

In [7]:
int global_var{-11};

In [8]:
int test2()
{
    return global_var;
}

In [9]:
test2()

(int) -11


# Internal and external linkage via the static and extern keywords

A variable with no linkage can only be referred to from the limited scope it exists in. Normal local variables are an example of variables with no linkage. Two local variables with the same name but defined in different functions have no linkage -- each will be considered an independent variable.

A variable with **internal linkage** is called an internal variable (or static variable). Variables with internal linkage can be used anywhere within the file they are defined in, but can not be referenced outside the file they exist in.

A variable with **external linkage** is called an external variable. Variables with external linkage can be used both in the file they are defined in, as well as in other files.
>By default:
 - non-const variables declared outside of a block are assumed to be external: `int x{2};`
 - const variables declared outside of a block are assumed to be internal: `const int y{3};`

In order to use an external global variable that has been declared in another file, you must use a variable forward declaration. For variables, creating a forward declaration is done via the **extern** keyword (with no initialization value).

To avoid redefining global constants each time they are included in a different file, define them as external variables and then utilize a forward declaration in a header file:

constants.cpp:

```c++
namespace Constants
{
    // actual global variables
    extern const double pi(3.14159);
    extern const double avogadro(6.0221413e23);
    extern const double my_gravity(9.2); // m/s^2 -- gravity is light on this planet
}
```

constants.h:

```c++
#ifndef CONSTANTS_H
#define CONSTANTS_H
 
namespace Constants
{
    // forward declarations only
    extern const double pi;
    extern const double avogadro;
    extern const double my_gravity;
}
 
#endif
```

Use in the code file stays the same:

```c++
#include "constants.h"
double circumference = 2 * radius * Constants::pi;
```

Here’s a summary chart of the use of the extern and static keywords for non-const and const variable use cases:

```c++
// Uninitialized definition:
int g_x;        // defines uninitialized global variable (external linkage)
static int g_x; // defines uninitialized static variable (internal linkage)
const int g_x;  // not allowed: const variables must be initialized
 
// Forward declaration via extern keyword:
extern int g_z;       // forward declaration for global variable defined elsewhere
extern const int g_z; // forward declaration for const global variable defined elsewhere
 
// Initialized definition:
int g_y(1);        // defines initialized global variable (external linkage)
static int g_y(1); // defines initialized static variable (internal linkage)
const int g_y(1);  // defines initialized static variable (internal linkage)
 
// Initialized definition w/extern keyword:
extern int g_w(1);       // defines initialized global variable (external linkage, extern keyword is redundant in this case)
extern const int g_w(1); // defines initialized const global variable (external linkage)
```

# Static Duration Variables
Using the static keyword on local variables changes them from automatic duration to static duration (also called fixed duration). A **static duration variable** (also called a “static variable”) is one that retains its value even after the scope in which it has been created has been exited! Static duration variables are only created (and initialized) once, and then they are persisted throughout the life of the program.

In [1]:
#include <iostream>
 
void incrementAndPrint()
{
    static int s_value = 1; // static duration via static keyword.  This line is only executed once.
    ++s_value;
    std::cout << s_value << std::endl;
} // s_value is not destroyed here, but becomes inaccessible


In [2]:
incrementAndPrint();
incrementAndPrint();
incrementAndPrint()

2
3
4


- Using `static` with a global variable affects its linkage (making it internally linked)
- Using `static` with a local variable affects its duration (giving it static duration)

# Summary

### Variable scope, duration, and linkage summary

Type  |	Example |	Scope  |	Duration  |	Linkage	| Notes
---|---
Local variable |	int x; |	Block scope |	Automatic duration |	No linkage	
Static local variable |	static int s_x; |	Block scope |	Static duration |	No linkage	
Dynamic variable |	int *x = new int; |	Block scope |	Dynamic duration |	No linkage	
Function parameter |	void foo(int x) |	Block scope |	Automatic duration |	No linkage	
External non-const global variable |	int g_x; |	File scope |	Static duration |	External linkage |	Initialized or uninitialized
Internal non-const global variable |	static int g_x; |	File scope |	Static duration |	Internal linkage |	Initialized or uninitialized
Internal const global variable |	const int g_x(1); |	File scope |	Static duration |	Internal linkage |	Must be initialized
External const global variable |	extern const int g_x(1); |	File scope |	Static duration |	External linkage |	Must be initialized

### Forward declaration summary

Type |	Example |	Notes
---|---
Function forward declaration |	void foo(int x); |	Prototype only, no function body
Non-const global variable forward declaration |	extern int g_x; |	Must be uninitialized
Const global variable forward declaration |	extern const int g_x; |	Must be uninitialized

# Namespaces 

Pretty straight-forward
```c++

namespace Foo
{
    // This doSomething() belongs to namespace Foo
    int doSomething(int x, int y)
    {
        return x + y;
    }
    
    namespace Zoo
    {
    // This doSomething() belongs to namespace Foo::Zoo
    int doSomething(int x, int y)
    {
        return x * y;
    }
    }
    
}

namespace Goo
{
    // This doSomething() belongs to namespace Foo
    int doSomething(int x, int y)
    {
        return x - y;
    }
}


int main(void)
{
    std::cout << Foo::doSomething(4, 3) << '\n'; \\ 4 + 3
    std::cout << Goo::doSomething(4, 3) << '\n'; \\ 4 - 3
    std::cout << Foo::Zoo::doSomething(4, 3) << '\n'; \\ 4 * 3
    return 0;
}
```

# The `using` Statement
- `using` as a declaration: `using std::cout` -> `from std import cout` (Python)
- `using` as a directive: `using std` -> `from std import *` (Python)

This respects the scoping of blocks:
```c++
int main()
{
    {
        using namespace Foo;
        // calls to Foo:: stuff here
    } // using namespace Foo expires
 
    {
        using namespace Goo;
        // calls to Goo:: stuff here
    } // using namespace Goo expires
 
    return 0;
}
```

# Implicit Type Coercion 

### Coericion Hierarchy

- long double (highest)
- double
- float
- unsigned long long
- long long
- unsigned long
- long
- unsigned int
- int (lowest)

# Explicit Type Coercion
### C-style casts (avoid this)
```c++
int i1 = 10;
int i2 = 4;
float f = (float)i1 / i2;
```

### static casts
```c++
int i1 = 10;
int i2 = 4;
float f = static_cast<float>(i1) / i2;
```

# Enumerated Types

In [1]:
enum Color
{
    COLOR_BLACK, // assigned 0
    COLOR_RED, // assigned 1
    COLOR_BLUE, // assigned 2
    COLOR_GREEN, // assigned 3
    COLOR_WHITE, // assigned 4
    COLOR_CYAN, // assigned 5
    COLOR_YELLOW, // assigned 6
    COLOR_MAGENTA // assigned 7
};
 
Color paint{COLOR_WHITE};

In [2]:
paint

(Color) (Color::COLOR_WHITE) : (unsigned int) 4


In [12]:
COLOR_WHITE < COLOR_YELLOW

(bool) true


## Enum Classes (C++ 11)
Using `enum class` the enumerator is not placed in the same scope as the enumeration itself. Instead, you access an enumerator using `::`, in the same way as specifying a namespace.

Nor will the compiler compare enumerators from **different** enumerations by falling back to their enum-value. Instead, you must cast the enumerator as an int.

In [1]:
// Define the enumeration `Animal` with the enumerators MOO & MEOW
enum class Animal
{
    MOO,
    MEOW
}

In [3]:
Animal moo{Animal::MOO};
Animal meow{Animal::MEOW}

(Animal) (Animal::MEOW) : (int) 1


In [4]:
moo < meow

(bool) true


In [5]:
static_cast<int>(moo)

(int) 0


## Typedefs & type aliases
**Typedefs** allow users to define an alias for a data type by using the keyword `typedef`.

In [6]:
typedef double distance_t  // `distance_t` is now an alias for the type `double`

Typedefs can be used for cross-platform programs, where `short` may be a 16-bit int on one platform, but `int` is 16-bit on another. This is how fixed width integers (like `int8_t`) are defined in C++11!

## Type Alias (C++11)
Type aliases help make typedefs more readable. Intstead of:

```cpp
typedef double distance_t
```

you can do

```cpp
using distance_t = double  // this is an overloaded use of `using` - unrelated to namespacing
```

## Structs
An **aggregate data type** is a data type that groups multiple individual variables together. One of the simplest aggregate data types is the **struct**. A struct (short for structure) allows us to group variables of mixed data types together into a single unit.

Struct definitions do not take any memory - you can simply put the definition in the header file and `#include` it anywhere. No need for forward declarations.

In [6]:
struct Particle
{
    float mass;
    int charge;
    bool spin;
}

In [7]:
Particle electron;
electron.mass = 1.0;
electron.charge = -1;
electron.spin = 1;

In [8]:
Particle proton{10.0, +1, 0};  // proton.spin=+1, etc.

In [9]:
Particle neutron;
neutron = {10.0, 0, 0};  // C++11 

### non-static member initialization (C++11/14}
You can give non-static default members an initial value:

In [2]:
struct Number
{
    int value = 0;
}

In C++11, you can't use the uniform-initialization syntax with members with intial values. You can in C++14.

In [3]:
Number num;
num.value

(int) 0


In [3]:
num.value

(int) 1


### Nested struct

In [10]:
struct Employee
{
    short id;
    int age;
    float wage;
};
 
struct Company
{
    Employee CEO; // Employee is a struct within the Company struct
    int numberOfEmployees;
};
 
Company myCompany = {{ 1, 42, 60000.0f }, 5 };